In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image


In [2]:
def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
            return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0]-box2[0])
            if dist > 4:
                continue
            
            iou_score = iou(box1[1:], box2[1:])
            
            if iou_score < 0.35:
                continue
            else:
                cost_matix[i,j]=0
    
    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

In [3]:
video_labels = pd.read_csv('/home/thinh/nfl/train_labels.csv')
video_labels = video_labels[video_labels['frame'] != 0].reset_index(drop=True)

In [4]:
test_lists = []

video_valid = ['57583_000082', '57586_004152', '57911_000147', '57997_003691', '57680_002206', '58095_004022', '57906_000718', '58005_001254', '57679_003316', '58103_003494', '57998_002181', '58048_000086']
for video_name in video_valid:
    test_lists.append(f'{video_name}_Endzone.mp4')
    test_lists.append(f'{video_name}_Sideline.mp4')

In [5]:
gt_df = video_labels[video_labels['video'].isin(test_lists) & (video_labels['impact']==1) & (video_labels['confidence']>1) & (video_labels['visibility']>0)]

In [6]:
score_threshold = 0.4
model_name = 'tito-checkpoint-D6-1024-A1-epoch022-fold0-gcp'
test_df = pd.read_csv(f'/home/thinh/nfl/effdet5-models/tito-1024/{model_name}.csv')
test_df = test_df[test_df['score'] > score_threshold]
test_df

,gameKey,playID,view,video,frame,left,width,top,height,score
548,57583,82,Endzone,57583_000082_Endzone.mp4,21,663,20,344,15,0.477444
603,57583,82,Endzone,57583_000082_Endzone.mp4,23,442,20,339,16,0.446461
910,57583,82,Endzone,57583_000082_Endzone.mp4,34,581,22,312,30,0.469096
936,57583,82,Endzone,57583_000082_Endzone.mp4,35,580,22,312,31,0.451447
962,57583,82,Endzone,57583_000082_Endzone.mp4,36,581,21,312,25,0.461582
...,...,...,...,...,...,...,...,...,...,...
282203,58103,3494,Sideline,58103_003494_Sideline.mp4,288,634,35,321,34,0.533941
282284,58103,3494,Sideline,58103_003494_Sideline.mp4,291,632,34,318,33,0.432139
282308,58103,3494,Sideline,58103_003494_Sideline.mp4,292,632,33,318,32,0.463433
282336,58103,3494,Sideline,58103_003494_Sideline.mp4,293,629,35,316,34,0.428844


In [7]:
gt_df['bot'] = gt_df['top'] + gt_df['height']
gt_df['right'] = gt_df['left'] + gt_df['width']
test_df['bot'] = test_df['top'] + test_df['height']
test_df['right'] = test_df['left'] + test_df['width']

/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/thinh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Frame filter

In [8]:
frame_df = pd.read_csv('../frame_impact_512.csv')
frame_df.head()

,label,frame,video
0,1,64,57787_003413_Endzone.mp4
1,1,193,58106_002918_Endzone.mp4
2,1,217,57913_000218_Sideline.mp4
3,1,25,58098_001193_Sideline.mp4
4,1,143,58005_001612_Endzone.mp4


In [9]:
test_df = pd.merge(test_df, frame_df, on=['video', 'frame']).drop(columns=['label'])
print(test_df.shape)

(2365, 12)


# Recent frame filter

In [10]:
# FILTER
#################
dropIDX = []
for keys in test_df.groupby(['gameKey', 'playID']).size().to_dict().keys():
    tmp_df = test_df.query('gameKey == @keys[0] and playID == @keys[1]')
    
    for index, row in tmp_df.iterrows():
        if row['view'] == 'Endzone':
            check_df = tmp_df.query('view == "Sideline"')
            if check_df['frame'].apply(lambda x: np.abs(x - row['frame']) <= 4).sum() == 0:
                dropIDX.append(index)
        
        if row['view'] == 'Sideline':
            check_df = tmp_df.query('view == "Endzone"')
            if check_df['frame'].apply(lambda x: np.abs(x - row['frame']) <= 4).sum() == 0:
                dropIDX.append(index)

In [11]:
test_df = test_df.drop(index = dropIDX).reset_index(drop = True)
test_df

,gameKey,playID,view,video,frame,left,width,top,height,score,bot,right
0,57583,82,Endzone,57583_000082_Endzone.mp4,34,581,22,312,30,0.469096,342,603
1,57583,82,Endzone,57583_000082_Endzone.mp4,35,580,22,312,31,0.451447,343,602
2,57583,82,Endzone,57583_000082_Endzone.mp4,36,581,21,312,25,0.461582,337,602
3,57583,82,Endzone,57583_000082_Endzone.mp4,37,582,20,311,25,0.442582,336,602
4,57583,82,Endzone,57583_000082_Endzone.mp4,40,442,21,320,18,0.432455,338,463
...,...,...,...,...,...,...,...,...,...,...,...,...
1774,58103,3494,Sideline,58103_003494_Sideline.mp4,179,682,14,283,13,0.410952,296,696
1775,58103,3494,Sideline,58103_003494_Sideline.mp4,181,749,13,278,18,0.409906,296,762
1776,58103,3494,Sideline,58103_003494_Sideline.mp4,184,739,13,281,16,0.403808,297,752
1777,58103,3494,Sideline,58103_003494_Sideline.mp4,185,732,16,281,17,0.455713,298,748


# Duplicate bbox

In [12]:
test_df.head(10)

,gameKey,playID,view,video,frame,left,width,top,height,score,bot,right
0,57583,82,Endzone,57583_000082_Endzone.mp4,34,581,22,312,30,0.469096,342,603
1,57583,82,Endzone,57583_000082_Endzone.mp4,35,580,22,312,31,0.451447,343,602
2,57583,82,Endzone,57583_000082_Endzone.mp4,36,581,21,312,25,0.461582,337,602
3,57583,82,Endzone,57583_000082_Endzone.mp4,37,582,20,311,25,0.442582,336,602
4,57583,82,Endzone,57583_000082_Endzone.mp4,40,442,21,320,18,0.432455,338,463
5,57583,82,Endzone,57583_000082_Endzone.mp4,41,442,21,320,18,0.491105,338,463
6,57583,82,Endzone,57583_000082_Endzone.mp4,42,440,22,319,18,0.539680,337,462
7,57583,82,Endzone,57583_000082_Endzone.mp4,42,279,22,320,16,0.411146,336,301
8,57583,82,Endzone,57583_000082_Endzone.mp4,43,417,22,311,31,0.590235,342,439
9,57583,82,Endzone,57583_000082_Endzone.mp4,43,440,21,319,18,0.573180,337,461


In [13]:
# Based on score
# Based on middle
# Based on last

# Check iou

dup_bboxes = []
test_df['bbox_id'] = test_df.apply(lambda x: f'{x.name}_{x.frame}', axis=1)

for video in test_df['video'].unique():
    tmp_df = test_df.query('video == @video')
    match_bbox_dict = {}
    check_ids = set()
    
    for index, row in tmp_df.iterrows():
        frame = row['frame']
        bbox = row[['left', 'top', 'right', 'bot']]
        bbox_id = row['bbox_id']
        if bbox_id in check_ids:
            continue
        
        while True:
            frame += 1
            next_tmp_df = tmp_df[tmp_df['frame'] == frame]
            if next_tmp_df.shape[0] == 0:
                break
            for next_i, next_row in next_tmp_df.iterrows():
                next_bbox = next_row[['left', 'top', 'right', 'bot']]
                next_bbox_id = next_row['bbox_id']
                
                if iou(bbox, next_bbox) > 0.5:
#                     if next_bbox_id in check_ids:
#                         print("ok")
                    check_ids.add(next_bbox_id)
                    if bbox_id not in match_bbox_dict: 
                        match_bbox_dict[bbox_id] = []
                    match_bbox_dict[bbox_id].append(next_bbox_id)
    
    for key, value in match_bbox_dict.items():
        new_value = [key] + value
        keep_key = new_value[int(len(new_value)/2 - 0.5)]
        new_value.remove(keep_key)
        dup_bboxes.extend(new_value)
#     print(match_bbox_dict)
#     break

In [14]:
# # Based on score
# # Based on middle
# # Based on last

# # Check iou

# keep_bboxes = []
# test_df['bbox_id'] = test_df.apply(lambda x: f'{x.name}_{x.frame}', axis=1)
# score_dict = dict(zip(test_df.bbox_id, test_df.score))

# for video in test_df['video'].unique():
#     tmp_df = test_df.query('video == @video')
#     match_bbox_dict = {}
#     check_ids = set()
    
#     for index, row in tmp_df.iterrows():
#         frame = row['frame']
#         bbox = row[['left', 'top', 'right', 'bot']]
#         bbox_id = row['bbox_id']
#         if bbox_id in check_ids:
#             continue
        
#         while True:
#             frame += 1
#             next_tmp_df = tmp_df[tmp_df['frame'] == frame]
#             if next_tmp_df.shape[0] == 0:
#                 break
#             for next_i, next_row in next_tmp_df.iterrows():
#                 next_bbox = next_row[['left', 'top', 'right', 'bot']]
#                 next_bbox_id = next_row['bbox_id']
                
#                 if iou(bbox, next_bbox) > 0.5:
# #                     if next_bbox_id in check_ids:
# #                         print("ok")
#                     check_ids.add(next_bbox_id)
#                     if bbox_id not in match_bbox_dict: 
#                         match_bbox_dict[bbox_id] = []
#                     match_bbox_dict[bbox_id].append(next_bbox_id)
    
#     for key, value in match_bbox_dict.items():
#         max_bbox_id = key
#         max_score = score_dict[key]
#         for bbox_id in value:
#             bbox_score = score_dict[bbox_id]
#             if bbox_score > max_score:
#                 max_bbox_id = bbox_id
#         keep_bboxes.append(max_bbox_id)
# #     print(match_bbox_dict)
# #     break

In [15]:
# print(len(keep_bboxes))
print(len(set(dup_bboxes)))
print(len(dup_bboxes))

1051
1080


In [16]:
test_df = test_df[~test_df['bbox_id'].isin(dup_bboxes)].reset_index(drop=True)
# test_df = test_df[test_df['bbox_id'].isin(keep_bboxes)].reset_index(drop=True)

In [17]:
test_df

,gameKey,playID,view,video,frame,left,width,top,height,score,bot,right,bbox_id
0,57583,82,Endzone,57583_000082_Endzone.mp4,35,580,22,312,31,0.451447,343,602,1_35
1,57583,82,Endzone,57583_000082_Endzone.mp4,42,440,22,319,18,0.539680,337,462,6_42
2,57583,82,Endzone,57583_000082_Endzone.mp4,44,277,22,318,17,0.469185,335,299,12_44
3,57583,82,Endzone,57583_000082_Endzone.mp4,45,422,19,312,25,0.475465,337,441,14_45
4,57583,82,Endzone,57583_000082_Endzone.mp4,47,434,19,316,18,0.517091,334,453,21_47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
723,58103,3494,Sideline,58103_003494_Sideline.mp4,179,682,14,283,13,0.410952,296,696,1774_179
724,58103,3494,Sideline,58103_003494_Sideline.mp4,181,749,13,278,18,0.409906,296,762,1775_181
725,58103,3494,Sideline,58103_003494_Sideline.mp4,184,739,13,281,16,0.403808,297,752,1776_184
726,58103,3494,Sideline,58103_003494_Sideline.mp4,185,732,16,281,17,0.455713,298,748,1777_185


# Remove last frames

In [18]:
# test_df = test_df[test_df['frame'] < 250]

In [19]:
# test_df = test_df[test_df['frame'] % 2 == 0]

# F1

In [20]:
ftp, ffp, ffn = [], [], []
for count, video in enumerate(set(gt_df['video'])):
    pred_boxes = test_df[test_df['video']==video][["frame",'left', 'top', 'right', 'bot']].to_numpy()
    gt_boxes = gt_df[gt_df['video']==video][["frame",'left', 'top', 'right', 'bot']].to_numpy()
    tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
    ftp.append(tp)
    ffp.append(fp)
    ffn.append(fn)

tp = np.sum(ftp)
fp = np.sum(ffp)
fn = np.sum(ffn)
precision = tp / (tp + fp + 1e-6)
recall =  tp / (tp + fn +1e-6)
f1_score = 2*(precision*recall)/(precision+recall+1e-6)
print(f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision:.4f}, RECALL: {recall:.4f}, F1 SCORE: {f1_score:.4f}')

TP: 128, FP: 600, FN: 270, PRECISION: 0.1758, RECALL: 0.3216, F1 SCORE: 0.2274


In [21]:
# 3-0.48944-0.4 TP: 69, FP: 274, FN: 380, PRECISION: 0.2012, RECALL: 0.1537, F1 SCORE: 0.1742
# 4-0.4 TP: 76, FP: 365, FN: 373, PRECISION: 0.1723, RECALL: 0.1693, F1 SCORE: 0.1708
# 6-0.51472-0.4 TP: 91, FP: 596, FN: 358, PRECISION: 0.1325, RECALL: 0.2027, F1 SCORE: 0.1602
# 8-0.57403-0.4 TP: 111, FP: 791, FN: 338, PRECISION: 0.1231, RECALL: 0.2472, F1 SCORE: 0.1643
# 8-0.57403-0.4 TP: 107, FP: 795, FN: 291, PRECISION: 0.1186, RECALL: 0.2688, F1 SCORE: 0.1646 (correct gt)
# 15-0.79221-0.4 TP: 132, FP: 1033, FN: 317, PRECISION: 0.1133, RECALL: 0.2940, F1 SCORE: 0.1636
# 15-0.79221-0.4 TP: 127, FP: 1038, FN: 271, PRECISION: 0.1090, RECALL: 0.3191, F1 SCORE: 0.1625 (correct gt)
# 15-0.79221-0.5 TP: 113, FP: 693, FN: 336, PRECISION: 0.1402, RECALL: 0.2517, F1 SCORE: 0.1801
# 15-0.79221-0.6 TP: 90, FP: 419, FN: 359, PRECISION: 0.1768, RECALL: 0.2004, F1 SCORE: 0.1879

# deim
# 512-2-0.54903-0.4 TP: 57, FP: 270, FN: 341, PRECISION: 0.1743, RECALL: 0.1432, F1 SCORE: 0.1572
# 512-6--0.4 TP: 69, FP: 290, FN: 329, PRECISION: 0.1922, RECALL: 0.1734, F1 SCORE: 0.1823
# 512-5--0.4 TP: 82, FP: 509, FN: 316, PRECISION: 0.1387, RECALL: 0.2060, F1 SCORE: 0.1658
# 512-7--0.4 TP: 128, FP: 747, FN: 270, PRECISION: 0.1463, RECALL: 0.3216, F1 SCORE: 0.2011
# 512-8--0.4 TP: 86, FP: 519, FN: 312, PRECISION: 0.1421, RECALL: 0.2161, F1 SCORE: 0.1715

# 512, D6, no deim, A1
# 10-0.4 TP: 58, FP: 156, FN: 340, PRECISION: 0.2710, RECALL: 0.1457, F1 SCORE: 0.1895
# 12-0.4 TP: 81, FP: 212, FN: 317, PRECISION: 0.2765, RECALL: 0.2035, F1 SCORE: 0.2344
# 13-0.4 TP: 62, FP: 238, FN: 336, PRECISION: 0.2067, RECALL: 0.1558, F1 SCORE: 0.1776
# 15-0.4 TP: 56, FP: 130, FN: 342, PRECISION: 0.3011, RECALL: 0.1407, F1 SCORE: 0.1918
# 18-0.4 TP: 76, FP: 289, FN: 322, PRECISION: 0.2082, RECALL: 0.1910, F1 SCORE: 0.1992

# 512, D6, no deim, A
# 13-0.4 TP: 46, FP: 154, FN: 352, PRECISION: 0.2300, RECALL: 0.1156, F1 SCORE: 0.1538
# 15-0.4 TP: 61, FP: 255, FN: 337, PRECISION: 0.1930, RECALL: 0.1533, F1 SCORE: 0.1709
# 18-0.4 TP: 69, FP: 269, FN: 329, PRECISION: 0.2041, RECALL: 0.1734, F1 SCORE: 0.1875


# no deim
# 768-2-0.4 TP: 75, FP: 241, FN: 374, PRECISION: 0.2373, RECALL: 0.1670, F1 SCORE: 0.1961
# 768-4-0.4 TP: 129, FP: 445, FN: 320, PRECISION: 0.2247, RECALL: 0.2873, F1 SCORE: 0.2522
# 768-4-0.4 TP: 127, FP: 447, FN: 271, PRECISION: 0.2213, RECALL: 0.3191, F1 SCORE: 0.2613 (correct gt) (0.2572, 0.2667)
# 768-4-0.5 TP: 42, FP: 63, FN: 356, PRECISION: 0.4000, RECALL: 0.1055, F1 SCORE: 0.1670 (correct gt)
# 768-5-0.4 TP: 127, FP: 612, FN: 322, PRECISION: 0.1719, RECALL: 0.2829, F1 SCORE: 0.2138 (0.2360, 0.2437)
# 768-5-0.5 TP: 64, FP: 130, FN: 385, PRECISION: 0.3299, RECALL: 0.1425, F1 SCORE: 0.1991
# 768-6-0.4 TP: 129, FP: 579, FN: 269, PRECISION: 0.1822, RECALL: 0.3241, F1 SCORE: (0.2333)
# 768-7-0.4 TP: 143, FP: 672, FN: 255, PRECISION: 0.1755, RECALL: 0.3593, F1 SCORE: (0.2358)

# no deim
# 1024-3-0.4 TP: 77, FP: 182, FN: 321, PRECISION: 0.2973, RECALL: 0.1935, F1 SCORE: 0.2344
# 1024-4-0.4 TP: 108, FP: 428, FN: 290, PRECISION: 0.2015, RECALL: 0.2714, F1 SCORE: 0.2313
# 1024-5-0.4 TP: 125, FP: 586, FN: 273, PRECISION: 0.1758, RECALL: 0.3141, F1 SCORE: 0.2254
# 1024-7-0.4 TP: 163, FP: 837, FN: 235, PRECISION: 0.1630, RECALL: 0.4095, F1 SCORE: 0.2332
# 1024-10-0.4 TP: 184, FP: 1173, FN: 214, PRECISION: 0.1356, RECALL: 0.4623, F1 SCORE: 0.2097

# Correct gt
# 768-6-0.4 TP: 137, FP: 728, FN: 261, PRECISION: 0.1584, RECALL: 0.3442, F1 SCORE: 0.2169
# 768-6-0.5 TP: 66, FP: 238, FN: 332, PRECISION: 0.2171, RECALL: 0.1658, F1 SCORE: 0.1880
# 768-7-0.4 TP: 151, FP: 877, FN: 247, PRECISION: 0.1469, RECALL: 0.3794, F1 SCORE: 0.2118
# 768-7-0.5 TP: 96, FP: 342, FN: 302, PRECISION: 0.2192, RECALL: 0.2412, F1 SCORE: 0.2297
# 768-10-0.4 TP: 159, FP: 933, FN: 239, PRECISION: 0.1456, RECALL: 0.3995, F1 SCORE: 0.2134
# 768-10-0.5 TP: 116, FP: 462, FN: 282, PRECISION: 0.2007, RECALL: 0.2915, F1 SCORE: 0.2377
# 768-10-0.6 TP: 66, FP: 185, FN: 332, PRECISION: 0.2629, RECALL: 0.1658, F1 SCORE: 0.2034
# 768-15-0.4 TP: 181, FP: 1181, FN: 217, PRECISION: 0.1329, RECALL: 0.4548, F1 SCORE: 0.2057
# 768-20-0.4 TP: 178, FP: 1229, FN: 220, PRECISION: 0.1265, RECALL: 0.4472, F1 SCORE: 0.1972
# 768-20-0.5 TP: 156, FP: 919, FN: 242, PRECISION: 0.1451, RECALL: 0.3920, F1 SCORE: 0.2118

# Albu
# 512-11-0.3 TP: 86, FP: 468, FN: 312, PRECISION: 0.1552, RECALL: 0.2161, F1 SCORE: 0.1807
# 512-13-0.4 TP: 49, FP: 189, FN: 349, PRECISION: 0.2059, RECALL: 0.1231, F1 SCORE: 0.1541
# 512-16-0.4 TP: 83, FP: 370, FN: 315, PRECISION: 0.1832, RECALL: 0.2085, F1 SCORE: 0.1951
# 512-17-0.4 TP: 78, FP: 287, FN: 320, PRECISION: 0.2137, RECALL: 0.1960, F1 SCORE: 0.2045
# 512-18-0.4 TP: 75, FP: 253, FN: 323, PRECISION: 0.2287, RECALL: 0.1884, F1 SCORE: 0.2066
# 512-20-0.4 TP: 67, FP: 251, FN: 331, PRECISION: 0.2107, RECALL: 0.1683, F1 SCORE: 0.1872
# 512-26-0.4 TP: 102, FP: 623, FN: 296, PRECISION: 0.1407, RECALL: 0.2563, F1 SCORE: 0.1817
# 512-26-0.5 TP: 47, FP: 139, FN: 351, PRECISION: 0.2527, RECALL: 0.1181, F1 SCORE: 0.1610

# Albu + Deim
# 512-7-0.4 TP: 2, FP: 14, FN: 396, PRECISION: 0.1250, RECALL: 0.0050, F1 SCORE: 0.0097
# 512-15-0.4 TP: 43, FP: 111, FN: 355, PRECISION: 0.2792, RECALL: 0.1080, F1 SCORE: 0.1558
# 512-16-0.4 TP: 63, FP: 179, FN: 335, PRECISION: 0.2603, RECALL: 0.1583, F1 SCORE: 0.1969
# 512-17-0.4 TP: 65, FP: 250, FN: 333, PRECISION: 0.2063, RECALL: 0.1633, F1 SCORE: 0.1823
# 512-20-0.4 TP: 43, FP: 137, FN: 355, PRECISION: 0.2389, RECALL: 0.1080, F1 SCORE: 0.1488
# 512-22-0.4 TP: 54, FP: 178, FN: 344, PRECISION: 0.2328, RECALL: 0.1357, F1 SCORE: 0.1714

# 1024, A1, D6
# 10-0.4 TP: 62, FP: 238, FN: 336, PRECISION: 0.2067, RECALL: 0.1558, F1 SCORE: 0.1776
# 12-0.4 TP: 71, FP: 179, FN: 327, PRECISION: 0.2840, RECALL: 0.1784, F1 SCORE: 0.2191 (0.2127, 0.2199)
# 14-0.4 TP: 83, FP: 223, FN: 315, PRECISION: 0.2712, RECALL: 0.2085, F1 SCORE: 0.2358 (0.2332)
# 15-0.4 TP: 79, FP: 158, FN: 319, PRECISION: 0.3333, RECALL: 0.1985, F1 SCORE: 0.2488 (0.2339, 0.2385)
# 16-0.4 TP: 83, FP: 230, FN: 315, PRECISION: 0.2652, RECALL: 0.2085, F1 SCORE: 0.2335 (cut frame 180 0.2364)
# 17-0.4 TP: 98, FP: 307, FN: 300, PRECISION: 0.2420, RECALL: 0.2462, F1 SCORE: 0.2441
# 18-0.4 TP: 126, FP: 455, FN: 272, PRECISION: 0.2169, RECALL: 0.3166, F1 SCORE: 0.2574 (0.2624)
# 19-0.4 TP: 150, FP: 671, FN: 248, PRECISION: 0.1827, RECALL: 0.3769, F1 SCORE: 0.2461 (0.2519)
# 20-0.4 TP: 145, FP: 509, FN: 253, PRECISION: 0.2217, RECALL: 0.3643, F1 SCORE: 0.2757 (0.2645, 0.2697)
# 21-0.4 TP: 133, FP: 446, FN: 265, PRECISION: 0.2297, RECALL: 0.3342, F1 SCORE: 0.2723 (0.2707, 0.2787)
# 22-0.4 TP: 137, FP: 457, FN: 261, PRECISION: 0.2306, RECALL: 0.3442, F1 SCORE: 0.2762 (0.2715, 0.2783)
# 23-0.4 TP: 130, FP: 453, FN: 268, PRECISION: 0.2230, RECALL: 0.3266, F1 SCORE: 0.2650 (0.2593)
# 24-0.4 TP: 147, FP: 583, FN: 251, PRECISION: 0.2014, RECALL: 0.3693, F1 SCORE: 0.2606 (0.2643)
# 25-0.4 TP: 138, FP: 496, FN: 260, PRECISION: 0.2177, RECALL: 0.3467, F1 SCORE: 0.2674 (0.2691)
# 26-0.4 TP: 147, FP: 556, FN: 251, PRECISION: 0.2091, RECALL: 0.3693, F1 SCORE: 0.2670 (0.2650)
# 27-0.4 TP: 150, FP: 604, FN: 248, PRECISION: 0.1989, RECALL: 0.3769, F1 SCORE: 0.2604 (0.2629)
# 28-0.4 TP: 138, FP: 526, FN: 260, PRECISION: 0.2078, RECALL: 0.3467, F1 SCORE: 0.2599
# 29-0.4 TP: 140, FP: 538, FN: 258, PRECISION: 0.2065, RECALL: 0.3518, F1 SCORE: 0.2602
# 30-0.4 TP: 138, FP: 496, FN: 260, PRECISION: 0.2177, RECALL: 0.3467, F1 SCORE: 0.2674